# BioExplorer - CCFv3
![](../bioexplorer_ccfv3_banner.png)

### Connect to back-end

In [ ]:
from bioexplorer import BioExplorer, Vector3, MovieMaker
from tqdm import tqdm

url = 'localhost:5000'
be = BioExplorer(url)
mm = MovieMaker(be)
core = be.core_api()

load_cells = True
load_meshes = False
cell_radius = 5.0 # Micrometters


### Load Atlas

In [ ]:
population_name = 'atlas_ccfv3a_averaged'

In [ ]:
cerebellum_ids = [
       512,      1025,       519,      1033,       528,      1041,      1049,
      1056,      1064,      1073,      1091,       846,        91, 589508455,
       989,      1143,      1144,      1145,       645,       912,       920,
       928,       936,       944,     10672,     10673,     10674,     10675,
     10676,     10677,       951,     10680,     10679,     10678,     10681,
     10682,       957,     10683,     10687,     10688,     10689,     10690,
     10691,     10692,     10684,     10686,       968,       976,     10705,
     10706,     10707,     10708,     10709,     10710,     10711,     10712,
     10713,       984,     10714,     10715,     10716,     10717,     10718,
       992,     10720,     10721,     10722,     10719,     10723,     10724,
     10725,     10728,      1001,     10726,     10727,     10729,     10730,
     10731,     10733,     10732,     10734,     10737,     10736,     10735,
      1007,      1017,     10685
]

cerebellum_granular_layer_ids = [10675, 10678, 10708, 10711, 10684, 10720, 10723, 10690, 10726, 10705, 10735, 10687, 10681, 10729, 10672, 10732]
cerebellum_molecular_layer_ids =  [10677, 10680, 10710, 10713, 10686, 10722, 10725, 10692, 10728, 10707, 10737, 10689, 10683, 10731, 10674, 10734]

olfactory_bulb_ids = [507, 212, 228, 236, 244, 220]

medula_ids = [
      773,       781,      1039,       789,      1048,        45,      1069,
      560,       307,        53,       568,        61,       576,       69,
      839,      1098,        76,        77,       83,      1107,       852,
      859,       607,        96,       354, 589508451,       101,       106,
      112,       370,       372,       887,       379,       640,      386,
      642,       903,       135,       136,       651,       395,       653,
      143,       659,       661,       666,       154,       161,       674,
      169,       682,       938,       939,       429,       177,       691,
      437,       185,       955,       445,       701,       193,       963,
      711,       970,       203,       202,       206,       207,       720,
      209,       978,       217,       222,       225,       995,       230,
      235,       765
]

hippocampus_ids = [391, 775, 782, 399, 766, 19, 790, 407, 415, 799, 423, 807, 431, 815, 438, 823, 1080, 446, 454, 10702, 463, 10703, 10704, 726, 471, 982, 734, 479, 486, 742, 632, 495, 751, 758, 375, 504, 382]

fibers_ids = [960, 1000, 1009, 396, 109, 698]
regions_to_ignore = [
    1557651848, # Cell excluded from region 1557651847 (x<540)
    1024543562, 2358040414, 3263488087, 2416897036,
    3034756217, 2165415682, 2614168502, 1842735199,
    3101970431, 1140764290, 3092369320, 1811993763]


In [ ]:
cerrebelum_filter = str(cerebellum_ids).replace('[','').replace(']','')
cerebellum_granular_layer_filter = str(cerebellum_granular_layer_ids).replace('[','').replace(']','')
cerebellum_molecular_layer_filter = str(cerebellum_molecular_layer_ids).replace('[','').replace(']','')
olfactory_bulb_filter = str(olfactory_bulb_ids).replace('[','').replace(']','')
medula_filter = str(medula_ids).replace('[','').replace(']','')
ignore_filter = str(fibers_ids + regions_to_ignore).replace('[','').replace(']','')

In [ ]:
status = be.reset_scene()

In [ ]:
import os
from sqlalchemy import create_engine
from sqlalchemy.orm import Session

db_host = os.getenv('DB_HOST')
db_name = os.getenv('DB_NAME')
db_user = os.getenv('DB_USER')
db_password = os.getenv('DB_PASSWORD')

db_connection_string = 'postgresql+psycopg2://%s:%s@%s:5432/%s' % (db_user, db_password, db_host, db_name)
print('Connection string: ' + db_connection_string)

engine = create_engine(db_connection_string)
conn = engine.connect()

In [ ]:
regions = list()
with Session(engine) as session:
    data = session.execute('select distinct(region_guid) from %s.cell' % population_name)
    for d in data.all():
        regions.append(int(d[0]))
regions = list(set(regions) - set(regions_to_ignore))

In [ ]:
status = be.start_model_loading_transaction()
status = core.set_application_parameters(image_stream_fps=0)
status = be.reset_scene()
for region in tqdm(regions):
    atlas_assembly_name = '%d' % region
    be.remove_assembly(atlas_assembly_name)
    atlas_assembly = be.add_assembly(atlas_assembly_name)
    atlas_model = be.add_atlas(
        assembly_name=atlas_assembly_name,
        population_name=population_name,
        load_cells=load_cells, load_meshes=load_meshes, cell_radius=cell_radius,
        region_sql_filter='guid=%d' % region,
        mesh_scale=Vector3(1, 1, 1)
    )
status = be.commit_model_loading_transaction()
status = core.set_application_parameters(image_stream_fps=20)

In [ ]:
region_colors = dict()

def hex_to_rgb(value):
    value = value.lstrip('#')
    lv = len(value)
    return tuple(int(value[i:i + lv // 3], 16) / 256.0 for i in range(0, lv, lv // 3))

with Session(engine) as session:
    data = session.execute('select guid, color_hex_triplet from %s.region' % population_name)
    for d in data.all():
        region_colors[int(d[0])] = hex_to_rgb(d[1])

In [ ]:
model_ids = be.get_model_ids()['ids']
for model_id in model_ids:
    k = 0
    material_ids = be.get_material_ids(model_id)['ids']
    palette = list()
    opacities = list()
    shading_modes = list()
    specular_exponents = list()
    refraction_indices = list()
    glossinesses = list()
    for material_id in material_ids:
        c = [1, 0, 0]
        m_id = material_id
        if m_id in region_colors:
            c = region_colors[m_id]
        alpha = [1.0, 1.0, 1.0]

        if material_id in olfactory_bulb_ids:
            value = 0.5 * float(k) / float(len(olfactory_bulb_ids))
            alpha[0] -= value
            alpha[1] -= value
            alpha[2] -= value
            k += 1

        shading_modes.append(be.shading_mode.NONE)
        glossinesses.append(1.0)
        specular_exponents.append(50.0)
        palette.append([alpha[0] * c[0], alpha[1] * c[1], alpha[2] * c[2]])
        
    be.set_materials(
        model_ids=[model_id], material_ids=material_ids,
        shading_modes=shading_modes, 
        opacities=opacities, specular_exponents=specular_exponents,
        refraction_indices=refraction_indices, glossinesses=glossinesses,
        diffuse_colors=palette, specular_colors=palette)

In [ ]:
background_color = [0.0, 0.0, 0.0]
status = core.set_renderer(
    head_light=True,
    current='advanced', background_color=background_color,
    subsampling=4, max_accum_frames=128)
params = core.AdvancedRendererParams()
params.gi_ray_length = 1e6
params.shadow_intensity = 1.0
params.soft_shadow_strength = 0.5
params.main_exposure = 1.5
params.max_ray_depth = 1
params.epsilon_multiplier = 50.0
params.use_hardware_randomizer = True
status = core.set_renderer_params(params)

In [ ]:
core.set_camera(
    current='perspective',
    orientation=[0.8744458160476791, 0.09151682620478456, -0.401564627390352, 0.25634943991934056],
    position=[-2565.3701241987646, -2607.8377454106976, -1731.3329308640486],
    target=[6733.589672442965, 4796.273454159725, 6016.635720470601],
)

In [ ]:
models = core.scene.models
centers = dict()

delta = 0.0001
for model in models:
    bounds = model['bounds']
    aabb_min = bounds['min']
    aabb_max = bounds['max']
    aabb_center = [0, 0, 0]
    for k in range(3):
        aabb_center[k] = (aabb_min[k] + aabb_max[k]) / 2.0
    centers[aabb_center[1]] = model['id']
sorted_centers = sorted(centers)

### Movie

In [ ]:
model_ids = be.get_model_ids()['ids']
nb_models = len(model_ids)
nb_models

In [ ]:
keys = [
    {
        'apertureRadius': 0.0,
        'direction': [0.6553724423068927, 0.5218272307970103, 0.5460616293637766],
        'focalDistance': 1.0,
        'origin': [-2565.3701241987646, -2607.8377454106976, -1731.3329308640486],
        'up': [0.36593474619134375, -0.8518192703488876, 0.37482754993752265]
    }
    ,
    {
        'apertureRadius': 0.0,
        'direction': [0.0, 1.0, -4.440892098500626e-16],
        'focalDistance': 1.0,
        'origin': [7075.004011154174, -8914.854648733952, 5699.994689941412],
        'up': [0.0, 4.440892098500626e-16, 1.0]
    }
]

double_keys = list()
for key in keys:
    double_keys.append(key)
    double_keys.append(key)
nb_frames_per_key = int(nb_models / (len(double_keys) - 1))
mm.build_camera_path(double_keys, nb_frames_per_key, nb_frames_per_key / 10)

In [ ]:
model_ids = be.get_model_ids()['ids']
for model_id in tqdm(model_ids):
    core.update_model(model_id, visible=False)

In [ ]:
output_folder = '/scratch/videos/atlas/%s/reconstruction/v1' % population_name
os.makedirs(output_folder, exist_ok=True)
k = 4
reversed_sorted_centers = list(reversed(sorted_centers))

'''Show models in Y increasing order'''
frame = 0
for sorted_center in tqdm(reversed_sorted_centers):
    model_id = centers[sorted_center]
    core.update_model(model_id, visible=True)
    mm.set_current_frame(frame)
    mm.create_snapshot(
        renderer='advanced', base_name='%05d' % frame,
        path=output_folder, samples_per_pixel=64, size=[3840, 2160]
    )
    frame += 1